## Linear Regression

A linear model makes a prediction by simply computing a weighted sum of the input features, plus a constant called the bias term (also called the intercept term)

$$\hat y = \theta_0 + \theta_1x_1 + \theta_2x_2 + \cdot \cdot \cdot + \theta_nx_n$$

$\hat y$ is the predicted value

$n$ is the number of features

$x_i$ is the $i^{th}$ feature value

$\theta_j$ is the $j^{th}$ model parameter

This can be written much more concisely using a vectorized form

$$\hat y = h_\theta(x) = \theta \cdot x$$

$\theta$ is the model’s parameter vector, containing the bias term $\theta_0$ and the feature weights $\theta_1$ to $\theta_n$

$x$ is the instance’s feature vector, containing $x_0$ to $x_n$, with $x_0$ always equal to 1

$\theta \cdot x$ is the dot product of the vectors $\theta$ and $x$, which is of course equal to $\theta_0x_0 + \theta_1x_1 + \cdot \cdot \cdot + \theta_nx_n$

$h_0$ is the hypothesis function, using the model parameters $\theta$

We first need a measure of how well (or poorly) the model fits the training data. In Chapter 2 we saw that the most common performance measure of a regression model is the Root Mean Square Error (RMSE). Therefore, to train a Linear Regression model, you need to find the value of $\theta$ that minimizes the RMSE

In practice, it is simpler to minimize the Mean Square Error (MSE) than the RMSE, and it leads to the same result (because the value that minimizes a function also minimizes its square root)

The MSE of a Linear Regression hypothesis $h_0$ on a training set X is calculated using

$$ MSE(X,h_\theta) = {1\over{m}} \sum_{i =1 }^m  $$